# Optimisation – Project 1

## Part 3.2
Median problem with weighted Euclidean distance

\begin{equation}
\min_{x \in \mathbb{R}} \sum_{i \in \mathcal{M}} v^i d_2(a^i, x)
\end{equation}

where $\mathcal{M} = \{1, \dots, m\}$ and $0 ≤ v^i \in \mathbb{R}, \; \forall i \in \mathcal{M}$.

We will here look into the Weiszfeld algorithm for solving the problem above.

## See overleaf for some of the solutions

## 4

In [115]:
import autograd.numpy as np
from autograd import grad
import time

## A solution for implementing an apropriate stopping criterion

In [221]:
def ourWeiszfeld(A, v, x_init, epsilon = 10**(-6)):
    """
    A = [a^1,..., a^i,..., a^m]^T   where   a^i = [a_1^i, a_2^i]
    Hence, A is a m times 2 matrix
    v = [v^1,..., v^i,..., v^m]^T   where   0<v^i \in \mathbb{R}, \forall i \in \mathcal{M} 
    epsilon > 0
    """
    m = A.shape[1]
    
    #############################
    # Test if some positions are optimal
    #############################
    
    # We define a test function to make the code leaner
    def testK(k):
        sum1 = 0
        sum2 = 0
        for i in range(m): 
            if i != k:
                dist = np.linalg.norm((A[:, k] - A[:, i])) # d_2(*,*) euclidean distance
                sum1 += v[i]* (A[:, k].item(0) - A[:, i].item(0))/(dist)
                sum2 += v[i]* (A[:, k].item(1) - A[:, i].item(1))/(dist)
                
        result = np.sqrt(sum1**2 + sum2**2)
        return (result <= v[k])

    
    for k in range(m): 
        # theorem 1 test if the theorem is fulfilled for a "k"
        if testK(k): # do we need to pass the k? should rather not?
            x_star = A[:, k] # could just return A[:, k], but this is clearer
            return(x_star)
    
    print(f"Test is conducted")
    x = x_init
    
    #############################
    # Functions for stopping ciretion
    #############################
    
    def sigma(x):
        dist = np.linalg.norm(x[:, None] - A, axis = 0)
        return np.max(dist)
    
    def f_d2(x):
        return (np.sum(np.multiply(v,np.linalg.norm(x[:, None] - A, axis = 0))))

    gradient = grad(f_d2)
    
    def stoping_criterion(x):
        norm_grad = np.linalg.norm(gradient(x))
        sig = sigma(x)
        return ((norm_grad * sig)/(f_d2(x) - norm_grad * sig))
    
    k = 0
    stop_crit = stoping_criterion(x)
    convergence = [[stop_crit, k]]
    
    while stop_crit > epsilon: # We have k = 50 here just to make the while loop stop
        k += 1
        # We print the value of the stopping criterion to see what happens
        
        for j in range(len(x)):
            enum = 0
            deno = 0
            
            for i in range(m):
                dist = np.linalg.norm((np.squeeze(np.array(A[:, i])) - x))
                enum += v[i]*A[:, i].item(j)/dist
                deno += v[i]/dist

            x[j] = enum/deno
            
        stop_crit = stoping_criterion(x)
        print(stop_crit)
        convergence.append([stop_crit, k]) 
        
    return [x, convergence]

## Gradient descent algorithm

In [222]:
def steepestDescent(x_init, obj_func, max_iter = 10000, threshold = 10**(-6)):
    
    # We create the gradient function for the objective function using autograd
    gradient = grad(obj_func)
    
    # Here we initialize
    i = 0
    x = np.array(x_init) 
    rho = 0.4
    c = 0.6
    diff = np.full((len(x), 1), 100) # initialize some value
    
    def sigma(x):
        dist = np.linalg.norm(x[:, None] - A, axis = 0)
        return np.max(dist)

    def stoping_criterion_steepest(x):
        norm_grad = np.linalg.norm(gradient(x))
        sig = sigma(x)
        return ((norm_grad * sig)/(obj_func(x) - norm_grad * sig))
    
    
    stop_crit = stoping_criterion(x)
    convergence = [[stop_crit, i]]
    
    while stop_crit > threshold and i < max_iter: 
        
        # To count the number of iterations
        i += 1
        a = 1
        grad_k = gradient(x)
        
        # I make a pk for convenience 
        pk = -grad_k
        
        # The following is backtracking
        while obj_func(x + a*pk) > obj_func(x) + a*c*np.transpose(grad_k) @ pk: # i.e. repeat until not true
            a = rho*a
        
        # The new x is stored
        x = x + a*pk
        
        stop_crit = stoping_criterion(x)
        print(stop_crit)
        convergence.append([stop_crit, i]) 
        
    return [x, convergence]

## Stopping criterion

In [223]:
def sigma(x):
    dist = np.linalg.norm(x[:, None] - A, axis = 0)
    return np.max(dist)
    
def f_d2(x):
    return (np.sum(np.multiply(v,np.linalg.norm(x[:, None] - A, axis = 0))))

gradient = grad(f_d2)

def stoping_criterion(x):
    norm_grad = np.linalg.norm(gradient(x))
    sig = sigma(x)
    return ((norm_grad * sig)/(f_d2(x) - norm_grad * sig))


## Finding initial value for the algorithms

In [224]:
def steepDesc(x_init, obj_func, max_iter = 1000, threshold = 10**(-7)):
    
    # We create the gradient function for the objective function using autograd
    gradient = grad(obj_func)
    
    # Here we initialize
    i = 0
    x = np.array(x_init) 
    rho = 0.4
    c = 0.6
    diff = np.full((len(x), 1), 100) # initialize some value

    while i < max_iter and diff.any() > threshold:
        
        # To count the number of iterations
        i += 1
        a = 1
        grad_k = gradient(x)
        
        # I make a pk for convenience 
        pk = -grad_k
        
        # The following is backtracking
        while obj_func(x + a*pk) > obj_func(x) + a*c*np.transpose(grad_k) @ pk: # i.e. repeat until not true
            a = rho*a
        
        # The new x is stored
        x = x + a*pk
        
        diff = np.abs(a*pk)
        
    return x

## Initialise values

In [225]:
m = 1000
A = np.random.random(size = (2, m))
# Values in the interval 0 to 100
A = 100 * A

# Initialise x
x_init = np.mean(A, axis = 1)

# Make normalised weights
v = np.random.rand(m)
v = v/np.sum(v)

In [226]:
#############################
# Finding a starting point
#############################

# choose a starting point x = [x_1, x_2]^T, can be found solving the median problem with ||\cdot||_2^2
def f_d2_squared_norm(x):
    return (np.sum(np.multiply(v,np.linalg.norm(x[:, None] - A, axis = 0)**2)))

# Solving the problem with squared Euclidean distance
#x_init = steepDesc(x_init, f_d2_squared_norm, threshold = 0.001)

In [227]:
x_init1 = x_init

stoping_criterion(x_init), x_init, x_init1

(0.07019104931752425,
 array([49.92206129, 48.8836407 ]),
 array([49.92206129, 48.8836407 ]))

In [228]:
start_steepest = time.time()
steepest_descent_result = steepestDescent(x_init, f_d2)
end_steepest = time.time()

0.0689872020190585
0.06780413767985682
0.06664145372098668
0.06549875669323445
0.06437566209907908
0.06327179422340183
0.06218678597257094
0.06112027872142899
0.06007192216760351
0.05904137419245876
0.05802830072790719
0.05703237562820759
0.05605328054579149
0.05509070481008375
0.05414434530822309
0.053213906366535106
0.052299099631584696
0.051399643949620225
0.05051526524323619
0.04964569638411573
0.048790677060780184
0.04794995364036058
0.04712327902352772
0.04631041249186203
0.04551111954711415
0.04472517174200375
0.04395234650241601
0.04319242694107783
0.04244520166303719
0.04171046456349743
0.04098801461879537
0.04027765567152453
0.039579196211007237
0.03889244915048886
0.038217231602569506
0.03755336465449315
0.03690067314497343
0.03625898544426385
0.035628133239154286
0.035007951324516756
0.034398277402920854
0.03379895189370689
0.03320981775273575
0.03263072030385144
0.03206150708288146
0.031502027694786404
0.03095213368435158
0.030411678420594022
0.02988051699485069
0.02935850

3.631542005938224e-05
3.570796843312388e-05
3.5110723057937466e-05
3.452351157031357e-05
3.394616453466032e-05
3.33785153927975e-05
3.282040041542814e-05
3.2271658653223145e-05
3.173213188967119e-05
3.1201664594221664e-05
3.068010387644277e-05
3.016729944070976e-05
2.9663103542312132e-05
2.916737094358191e-05
2.8679958870988514e-05
2.8200726973569272e-05
2.7729537281090917e-05
2.7266254163466843e-05
2.6810744290738866e-05
2.636287659437067e-05
2.5922522227685643e-05
2.5489554529211574e-05
2.5063848984233193e-05
2.464528318897144e-05
2.423373681414999e-05
2.382909156943889e-05
2.343123116949451e-05
2.3040041298870543e-05
2.265540957897904e-05
2.2277225534874722e-05
2.1905380562940442e-05
2.153976789882883e-05
2.118028258616701e-05
2.0826821445857567e-05
2.047928304561973e-05
2.0137567670451604e-05
1.9801577293163616e-05
1.9471215545805675e-05
1.9146387691266876e-05
1.8827000595714356e-05
1.8512962701205773e-05
1.820418399845776e-05
1.7900576001357984e-05
1.7602051720078858e-05
1.7308525

In [229]:
start_weiszfeld = time.time()
weiszfeld_result = ourWeiszfeld(A, v, x_init)
end_weiszfeld = time.time()

Test is conducted
0.03436255153499734
0.016889481794546208
0.008422342721387952
0.004246167291096483
0.0021579890699300565
0.0011040550542673968
0.0005680938583543247
0.0002937620078611575
0.00015254834080366073
7.95014634053155e-05
4.155741824990801e-05
2.1777546139626556e-05
1.1435830458979262e-05
6.015401810329813e-06
3.168584017956567e-06
1.670927615101124e-06
8.819615652990064e-07


In [219]:
x_init, x_init1, steepest_descent_result[0], weiszfeld_result[0], A

(array([51.81878281, 48.11632403]),
 array([51.81878281, 48.11632403]),
 array([51.81878348, 48.1163198 ]),
 array([51.81878281, 48.11632403]),
 array([[13.26727735, 54.10634716, 24.45660386, ..., 84.90817699,
         40.42216787, 91.16732583],
        [16.42720357, 26.5628701 , 18.23332091, ..., 45.5495367 ,
         11.76849191, 60.77515691]]))

In [220]:
time_steep = end_steepest - start_steepest
time_weiszfeld = end_weiszfeld - start_weiszfeld

time_steep, time_weiszfeld

(0.9179761409759521, 15.812547206878662)

In [198]:
stoping_criterion(steepest_descent_result[0]), steepest_descent_result

(9.989567791611594e-07,
 [array([19.10989323, 28.04886279]),
  [[0.7327742605727734, 0],
   [0.7189311957980061, 1],
   [0.7054072910489543, 2],
   [0.6921930317536076, 3],
   [0.6792791979251321, 4],
   [0.6666568662523163, 5],
   [0.6543174102289983, 6],
   [0.642252498565745, 7],
   [0.630454092111587, 8],
   [0.6189144394973904, 9],
   [0.6076260716959826, 10],
   [0.5965817956777276, 11],
   [0.585774687324129, 12],
   [0.5751980837464759, 13],
   [0.5648455751416099, 14],
   [0.5547109963027738, 15],
   [0.5447884178902056, 16],
   [0.5350721375537396, 17],
   [0.5255566709881754, 18],
   [0.5162367429915897, 19],
   [0.5071072785870393, 20],
   [0.49816339425926, 21],
   [0.48940038934991636, 22],
   [0.48081373764772783, 23],
   [0.472399079203233, 24],
   [0.4641522123921648, 25],
   [0.4560690862461741, 26],
   [0.44814579306505714, 27],
   [0.4406087838559075, 28],
   [0.43356729191143606, 29],
   [0.42663774741694566, 30],
   [0.41981856817719715, 31],
   [0.413108202101252

### From the what we see here, the stopping criterion seem to work as well as the steepest descent algorithm

In [25]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull, convex_hull_plot_2d   # get a convex hull computation algorithm
 
 
 
#points = np.random.rand(10, 2)   # 30 random points in 2-D
#hull = ConvexHull(points)
 
#vecs = []
#for i in range(len(hull.vertices)):
 ##   element = points[hull.vertices[i]]
 #   element = element.tolist()
 #   vecs.append(element)
 
def distance(x_1,x_2):
    x_1 = np.array(x_1)
    x_2 = np.array(x_2)
    diff = x_1 - x_2
    return np.linalg.norm(diff, 2) 
 
 
def isCounterClockwise(n1,n2, n3):
    return ((n1 <n2 <n3) or (n3 < n1 < n2) or (n2 < n3 < n1))
 
def Area(vtxList, n1, n2, n3): # get area of triangle made of point 1,2, 3 from a list of vertices
    n = len(vtxList)
    vtx1 = vtxList[n1]
    vtx2 = vtxList[n2]
    vtx3 = vtxList[n3]
    # print(vtx1, vtx2, vtx3)
    a = distance(vtx1,vtx2)
    b = distance(vtx3,vtx2)
    c = distance(vtx1,vtx3)
    s = (a + b + c) / 2
    area = (s*(s-a)*(s-b)*(s-c)) ** 0.5
    if isCounterClockwise(n1,n2,n3):
        return round(area,3)
    else:
        return round(-area,3)
 
 
def GetAllAntiPodalPairs(vtxList):
    pairList = []
    n = len(vtxList)
    i_1 = 1
    i_n = 0
    i = i_n
    j = (i + 1)%n
    while (Area(vtxList,i, (i + 1)%n, (j + 1)%n) > Area(vtxList, i, (i + 1)%n, j)):
        j = (j + 1)%n
        j0 = j
    while (j != i_1):
        i = (i + 1)%n
        pairList.append([i, j])
        while ((Area(vtxList, i, (i + 1)%n, (j + 1)%n) > Area(vtxList, i, (i + 1)%n, j))):
            j = (j + 1)%n
            if (j != i_1):
                pairList.append([i, j])
        if (Area(vtxList, i, (i + 1)%n, (j + 1)%n) == Area(vtxList, i, (i + 1)%n, j)):
            if ((i, j) != (j0, i_n)):
                pairList.append([i, (j + 1)%n])
 
    return(pairList)
 
 
 
def findDiameter(polygon):
    maxx = 0
    antiPodalPairs = GetAllAntiPodalPairs(polygon)
    for pair in antiPodalPairs:
        pairPoint1 = polygon[pair[0]]
        pairPoint2 = polygon[pair[1]]
        candidate = distance(pairPoint1, pairPoint2)
        if (candidate > maxx):
            maxx = candidate
    return maxx
 
 
# EXAMPLE
 
polygonZ = [[0,0], [3,0], [2,1], [0,1]]
print(findDiameter(polygonZ))

3.1622776601683795


## 5

Compare using precision, speed and possibly stability
